In [ ]:
# 第 10 章 序列的修改、散列和切 片

In [1]:
# 示例 10-2 vector_v1.py:从 vector2d_v1.py 衍生而来  示例 10-2 是第 1 版 Vector 类的实现代码(以示例 9-2 和示例 9-3 中的 代码为基础)
from array import array
import reprlib
import math


class Vector:
    typecode = 'd'
    
    def __init__(self, components):
        # self._components 是“受保护的”实例属性，把 Vector 的分量保存 在一个数组中
        self._components = array(self.typecode, components)
        
    def __iter__(self):
        # 为了迭代，我们使用 self._components 构建一个迭代器
        return iter(self._components)
    
    def __repr__(self):
        # 使用 reprlib.repr() 函数获取 self._components 的有限长度表 示形式(如 array('d', [0.0, 1.0, 2.0, 3.0, 4.0, ...]))
        components = reprlib.repr(self._components)
        # 把字符串插入 Vector 的构造方法调用之前，去掉前面的 array('d' 和后面的 )
        components = components[components.find('['):-1]
        return f'Vector({components})'
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        # 直接使用 self._components 构建 bytes 对象。
        return (bytes([ord(self.typecode)]) + bytes(self._components))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        # 不能使用 hypot 方法了，因此我们先计算各分量的平方之和，然后再使用 sqrt 方法开平方。
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)  # 我们只需在 Vector2d.frombytes 方法的基础上改动最后一行:直接把 memoryview 传给构造方法，不用像前面那样使用 * 拆包。
    

In [3]:
# 10.3 协议和鸭子类型
"""
在第 1 章我们就说过，在 Python 中创建功能完善的序列类型无需使用继
承，只需实现符合序列协议的方法。不过，这里说的协议是什么呢?
在面向对象编程中，协议是非正式的接口，只在文档中定义，在代码中 不定义。例如，Python 的序列协议只需要 __len__ 和 __getitem__ 两 个方法。
任何类(如 Spam)，只要使用标准的签名和语义实现了这两 个方法，就能用在任何期待序列的地方。Spam 是不是哪个类的子类无 关紧要，只要提供了所需的方法即可。
示例 1-1 中见过一例，这里再次 给出代码，
如示例 10-3 所示。
"""

# 示例 10-3 示例 1-1 的代码，为了方便，再次给出
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]